In [101]:
import spacy
import pandas as pd
import re

In [163]:
data = pd.read_csv('pledges_prepare.csv', delimiter=',')

In [73]:
bag_of_words = {}
for index, row in data.iterrows():
    row = row['vehicleproperty_description_short']
    row = row.lower()
    row = re.sub(r'\W', ' ', row)
    row = re.sub(r'\d', ' ', row)
    row = re.sub(r'\s+', ' ', row)
    words = row.split(' ')
    for word in words: 
        if word in bag_of_words and len(word) > 1:
            bag_of_words[word] = bag_of_words[word] + 1
        else: 
            bag_of_words[word] = 1
# print(bag_of_words)
pd_bag = pd.DataFrame.from_dict(bag_of_words, orient='index', columns=[ 'count'])
pd_bag_sort = pd_bag[pd_bag['count']>1].sort_values('count', ascending=False)

pd_bag_sort.to_csv('data/bag_of_words.csv')
pd_bag_sort.count()

count    3385
dtype: int64

In [99]:
# bag_of_words_clean = pd.read_csv('data/bag_of_words_clean.csv')
# bag_of_words_clean_unique = pd.DataFrame(set(bag_of_words_clean['words'].values))
# bag_of_words_clean_unique.to_csv('data/bag_of_words_clean_unique.csv',  index=False)
bag_of_words_clean_unique = pd.read_csv('data/bag_of_words_clean_unique.csv')
bag_of_words_uniq = {}
for index,word in bag_of_words_clean_unique.iterrows():
    word = word['word']
    if word in bag_of_words:
        bag_of_words_uniq[word] = bag_of_words[word]
bag_of_words_uniq['klas']

1

In [94]:
import wordsegment
wordsegment.load()

In [100]:
from collections import Counter

bag_of_words_clean_unique = pd.read_csv('data/bag_of_words_clean_unique.csv')
wordsegment.UNIGRAMS.clear()
wordsegment.UNIGRAMS.update(bag_of_words_uniq)

# def pairs(iterable):
#     iterator = iter(iterable)
#     values = [next(iterator)]
#     for value in iterator:
#         values.append(value)
#         yield ' '.join(values)
#         del values[0]

# wordsegment.BIGRAMS.clear()
# wordsegment.BIGRAMS.update(Counter(pairs(tokenize(text))))
from wordsegment import _segmenter

def identity(value):
    return value

_segmenter.clean = identity
wordsegment.segment('Экскаватор ЕК 14-21год выпкска 2010заводской')

['Экскаватор ЕК 14-21год в', 'ыпкска 2010заводской']

In [164]:
nlp = spacy.load("ru_core_news_sm")

config = { "overwrite_ents": True, "validate": True }
ruler = nlp.add_pipe("entity_ruler", before="ner", config=config).from_disk("./data/rules.jsonl", )

ents = []
for article in data['with_space'].values:
    article = str(article)
    doc = nlp(article)
    list_brands = [ent.ent_id_ for ent in doc.ents if ent.label_ == 'BRAND']
    ents.append(None if len(list_brands) == 0 else list_brands[0])

df2 = data.assign(Brand_NER=ents)

In [165]:
df2['Brand_NER'].value_counts()

lada             754
mercedes-benz    594
toyota           449
kia              432
hyundai          392
                ... 
kaessbohrer        1
dacia              1
asia               1
jmc                1
peterbilt          1
Name: Brand_NER, Length: 118, dtype: int64

In [166]:
# df2.to_csv('with_ner.csv', index=False)

In [167]:
df2['Brand_NER'].isnull().value_counts()

False    7993
True     2007
Name: Brand_NER, dtype: int64

In [169]:
df3 = df2[df2['Brand_NER'].isnull()][df2['vehicleproperty_description_short'] != 'Автомобиль'][150:200]
df3

/var/folders/2t/mb7xtxlx289fhmxl983vql2c0000gp/T/ipykernel_53270/1294924689.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df3 = df2[df2['Brand_NER'].isnull()][df2['vehicleproperty_description_short'] != 'Автомобиль'][150:200]


,id,notificationreferencenumber,vehicleproperty_vin,vehicleproperty_description_short,nBody,nChas,BrandModel,BrandModel2,BrandModel3,Brand,Brand2,with_space,Brand_NER
1224,4057,2014-000-001291-616,ZBAS36R2M35022895,Полуприцеп-цистерна PLACENZA S36R2M351991 г.в....,NaN,NaN,NaN,NaN,IZH OTHER,PIACENZA,NaN,Полуприцеп-цистерна PLACENZA S36R2M351991 г.в....,None
1233,4075,2014-000-001301-451,XW8LB65L2EH702430,Легковой автомобиль SKODAYETI Темно-синий,NaN,NaN,NaN,NaN,NaN,AUDI,NaN,Легковой автомобиль SKODAYETI Темно-синий,None
1258,4100,2014-000-001323-380,Z94CU51DBFR105374,SOLARIS БЕЛЫЙ,NaN,NaN,NaN,NaN,NaN,HYUNDAI,NaN,SOLARIS БЕЛЫЙ,None
1262,4104,2014-000-001330-440,XWEFZ417BE0000462,YD (CeratoForte) СЕРЕБРИСТЫЙ,NaN,NaN,NaN,NaN,NaN,GENESIS,NaN,YD (CeratoForte) СЕРЕБРИСТЫЙ,None
1283,4129,2014-000-001353-051,NaN,Асфальтоукладчик VOGELE SUPER 1600-2цвет зелен...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Асфальтоукладчик VOGELE SUPER 1600-2цвет зелен...,None
1306,4307,2014-000-001379-441,Z9G438921D0000554,автомобиль Рефрижератор 4389212013 года выпуск...,NaN,NaN,NaN,NaN,IZH OTHER,HINO,NaN,автомобиль Рефрижератор 4389212013 года выпуск...,None
1307,4308,2014-000-001380-275,NaN,VINотсутствуетнаименование (тип ТС)Полуприцеп ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VINотсутствуетнаименование (тип ТС)Полуприцеп ...,None
1313,4315,2014-000-001398-137,X0T952340C0000754,ПОЛУПРИЦЕП-САМОСВАЛ ПТС 50 НР 202436МАРКАМОДЕЛ...,NaN,NaN,NaN,NaN,NaN,GAZ,NaN,ПОЛУПРИЦЕП-САМОСВАЛ ПТС 50 НР 202436МАРКАМОДЕЛ...,None
1315,4317,2014-000-001400-394,WDC1660571A132685,ЛЕГКОВОЙ МЕРСЕДЕС ПТС 77 УВ 872495МАРКАМОДЕЛЬ-...,NaN,NaN,NaN,MERCEDES-BENZ OTHER,MERCEDES-BENZ OTHER,MERCEDES-BENZ,NaN,ЛЕГКОВОЙ МЕРСЕДЕС ПТС 77 УВ 872495МАРКАМОДЕЛЬ-...,None
1318,4321,2014-000-001407-374,NaN,Автопогрузчик UTILEV UT15P; заводской № машины...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Автопогрузчик UTILEV UT15P; заводской № машины...,None


In [157]:
df3.loc[1219]['vehicleproperty_description_short']

'Полуприцеп-самосвал Тонар 952312008 г.в. ПТС: 50 МР 008484выдан 17.10.2008 г.ООО МЗ Тонар государственный регистрационный знак: ВВ 1056 52'

In [ ]:

patterns = [

{"label": "BRAND", "pattern": [{"LOWER": "маз"}, {"IS_PUNCT": True},{'IS_DIGIT': True}], "id": "maz"},
{"label": "BRAND", "pattern": [{"LOWER": {"REGEX": "^маз-[0-9]*$"}}], "id": "maz"},
{"label": "BRAND", "pattern": [{"LOWER": "john"}, {"LOWER": "deere"}], "id": "john deere"}
]
ruler.add_patterns(patterns)

doc = nlp('Модель автомобиля: CHEVROLETKLAN (J200/Chevrol')
print(doc.ents)
print([ent.ent_id_ for ent in doc.ents])
print([(token.lemma_, token.tag_) for token in doc])